In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
plays = pd.read_csv('data/user_artists.dat', sep='\t')
artists = pd.read_csv('data/artists.dat', sep='\t', usecols=['id','name'])

# Merge artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [4]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)

# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
#print(ap)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values

# Show sparsity
density = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("density: %.2f" % density)

density: 0.28


In [152]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


array([   2,    3,    4, ..., 2097, 2099, 2100])

In [7]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo()
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

/Users/buu/venv/base/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [8]:
# Train
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10, num_threads=2)

In [9]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.37, test 0.13.
AUC: train 0.96, test 0.85.


In [10]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['Depeche Mode' 'Madonna' 'Pet Shop Boys' ... 'ブラック・キャッツ' 'Luctus'
 'The Shape Shifters']


In [161]:
model2 = LightFM(learning_rate=0.08, loss='warp')
model2.fit(train, epochs=10, num_threads=2)

#### Try some parametter and get a board with better

In [166]:
# Evaluate
trp = precision_at_k(model, train, k=10).mean()
tep = precision_at_k(model, test, k=10, train_interactions=train).mean()

tra = auc_score(model, train).mean()
tea = auc_score(model, test, train_interactions=train).mean()

dico = {'Loss':'Warp', 'K':10, 'Learning_rate':0.08, 'Train Precision':trp, 'Test Precision':tep, 'Train Auc':tra,'Test Auc':tea}

tab = pd.DataFrame(dico.items(), columns=['Parameter', 'Values'])

In [167]:
tab

,Parameter,Values
0,Loss,Warp
1,K,10
2,Learning_rate,0.08
3,Train Precision,0.367922
4,Test Precision,0.129632
5,Train Auc,0.960784
6,Test Auc,0.852505


In [180]:

def recommend(user_id):
    scores = model.predict(user_id, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    reco = pd.DataFrame(top_items[:10], columns=['Recommandations'])
    return reco

In [182]:
recommend(8)

,Recommandations
0,Radiohead
1,The Beatles
2,Coldplay
3,Muse
4,Sigur Rós
5,Kings of Leon
6,Arctic Monkeys
7,The Killers
8,The Cure
9,Joy Division


In [108]:
#  get_ground_truth qui renvoie les artistes ecoutés par un utilisateur par ordre décroissant du playCountScaled